In [1]:
# !conda install -y gdown
# print('hi')
!git clone https://github.com/mattjhayes3/cs234-project.git
%cd /kaggle/working/cs234-project
!pip install wandb
import wandb

wandb.login(key="KEY")
# wandb.init()

from huggingface_hub import notebook_login,login

# notebook_login("KEY")
login("KEY")

# !gdown --id 1TTg8s_dj60EKl4No2unSvYmMyMopPVJ6
# !gdown --id 1Z7HvAokBQu65jew4ou2DjOYRi23OrJdr
!mkdir results
!pip install datasets>=1.17.0 torch>=1.4.0 tqdm transformers accelerate peft>=0.3.0 tyro>=0.5.7
!pip install git+https://github.com/mattjhayes3/trl.git

Cloning into 'cs234-project'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 111 (delta 37), reused 98 (delta 27), pack-reused 0
Receiving objects: 100% (111/111), 23.60 MiB | 21.04 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (46/46), done.
Encountered 2 file(s) that should have been pointers, but weren't:
	pref_pairs_16_token_tokenized_split/data-00000-of-00002.arrow
	pref_pairs_16_token_tokenized_split/data-00001-of-00002.arrow
/kaggle/working/cs234-project


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
  Cloning https://github.com/mattjhayes3/trl.git to /tmp/pip-req-build-lntrw15o
  Running command git clone --filter=blob:none --quiet https://github.com/mattjhayes3/trl.git /tmp/pip-req-build-lntrw15o
  Resolved https://github.com/mattjhayes3/trl.git to commit 0f327a2feb6a1bba0c9494205875bea5c868ed75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.8.7.dev0-py3-none-any.whl size=209518 sha256=7836afd7f1a65a1de4fab2077bd1046c371d9242cf64de6d982bd1fafd6e5fe9
  Stored in directory: /tmp/pip-ephem-wheel-cache-m75tf_q_/wheels/b5/f5/74/f982e27bdb1c23b205f8bfcaaed174cf3d2c06bd1a5f5926cc
Successfully built trl


In [4]:
! python dpo.py  --output_dir=test_dpo_1_3_lr2e_6  --beta=1.3 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_tokenized_split     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 150     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=500 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch'

2024-05-24 01:15:13.055377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 01:15:13.055446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 01:15:13.056966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [6]:
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_dpo_1_3_lr2e_6-2024.05.24.01.15/checkpoint-3327
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_dpo_1_3_lr2e_6-2024.05.24.01.15/checkpoint-2218
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_dpo_1_3_lr2e_6-2024.05.24.01.15/checkpoint-1109

2024-05-24 02:21:06.404006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 02:21:06.404054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 02:21:06.405314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
PPOConfig: PPOConfig(exp_name='eval', dry_run=False, eval_model='./test_dpo_1_3_lr2e_6-2024.05.24.01.15/checkpoint-3327', seed=0, log_with='wandb', task_name=None, model_name='lvwerra/gpt2-imdb', query_dataset='imdb', reward_model='sentiment-analysis:siebert/sentiment-roberta-large-english', remove_unused_columns=True, tracker_kwargs={'wandb': {'name': 'eval-2024

In [8]:
! python dpo.py  --loss_type=ipo --output_dir=test_ipo_0_2_lr2e_6  --beta=0.2 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_tokenized_split     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 150     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=500 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch'

2024-05-24 02:39:34.889220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 02:39:34.889274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 02:39:34.890834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [9]:
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_2_lr2e_6-2024.05.24.02.39/checkpoint-3327
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_2_lr2e_6-2024.05.24.02.39/checkpoint-2218
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_2_lr2e_6-2024.05.24.02.39/checkpoint-1109

2024-05-24 03:49:52.831518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 03:49:52.831573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 03:49:52.833205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
PPOConfig: PPOConfig(exp_name='eval', dry_run=False, eval_model='./test_ipo_0_2_lr2e_6-2024.05.24.02.39/checkpoint-3327', seed=0, log_with='wandb', task_name=None, model_name='lvwerra/gpt2-imdb', query_dataset='imdb', reward_model='sentiment-analysis:siebert/sentiment-roberta-large-english', remove_unused_columns=True, tracker_kwargs={'wandb': {'name': 'eval-2024

In [10]:
! python dpo.py  --loss_type=ipo --output_dir=test_ipo_0_4_lr2e_6  --beta=0.4 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_tokenized_split     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 150     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=500 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch'

2024-05-24 04:22:13.072404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 04:22:13.072462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 04:22:13.074088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [17]:
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_4_lr2e_6-2024.05.24.04.22/checkpoint-3327
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_4_lr2e_6-2024.05.24.04.22/checkpoint-2218
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test_ipo_0_4_lr2e_6-2024.05.24.04.22/checkpoint-1109

2024-05-24 05:33:28.357184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 05:33:28.357233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 05:33:28.358604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
PPOConfig: PPOConfig(exp_name='eval', dry_run=False, eval_model='./test_ipo_0_4_lr2e_6-2024.05.24.04.22/checkpoint-3327', seed=0, log_with='wandb', task_name=None, model_name='lvwerra/gpt2-imdb', query_dataset='imdb', reward_model='sentiment-analysis:siebert/sentiment-roberta-large-english', remove_unused_columns=True, tracker_kwargs={'wandb': {'name': 'eval-2024